In [95]:
import json
import csv
import pandas as pd
import requests
import pickle
from pandas.io.json import json_normalize

In [6]:
filename = './trip-data/sample.csv'
csvfile = open(filename, 'r')
reader = csv.reader(csvfile)
keys  = reader.next()
print keys

['tripduration', 'starttime', 'stoptime', 'start station id', 'start station name', 'start station latitude', 'start station longitude', 'end station id', 'end station name', 'end station latitude', 'end station longitude', 'bikeid', 'usertype', 'birth year', 'gender']


In [9]:
data = pd.read_csv(filename)
data.head(10)

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,461,2/1/2016 00:00:08,2/1/2016 00:07:49,480,W 53 St & 10 Ave,40.766697,-73.990617,524,W 43 St & 6 Ave,40.755273,-73.983169,23292,Subscriber,1966,1
1,297,2/1/2016 00:00:56,2/1/2016 00:05:53,463,9 Ave & W 16 St,40.742065,-74.004432,380,W 4 St & 7 Ave S,40.734011,-74.002939,15329,Subscriber,1977,1
2,280,2/1/2016 00:01:00,2/1/2016 00:05:40,3134,3 Ave & E 62 St,40.763126,-73.965269,3141,1 Ave & E 68 St,40.765005,-73.958185,22927,Subscriber,1987,1
3,662,2/1/2016 00:01:00,2/1/2016 00:12:02,537,Lexington Ave & E 24 St,40.740259,-73.984092,428,E 3 St & 1 Ave,40.724677,-73.987834,20903,Subscriber,1983,2
4,355,2/1/2016 00:01:41,2/1/2016 00:07:36,284,Greenwich Ave & 8 Ave,40.739017,-74.002638,521,8 Ave & W 31 St,40.750967,-73.994442,23228,Subscriber,1978,1
5,1034,2/1/2016 00:01:44,2/1/2016 00:18:58,223,W 13 St & 7 Ave,40.737815,-73.999947,305,E 58 St & 3 Ave,40.760958,-73.967245,23761,Subscriber,1966,1
6,601,2/1/2016 00:02:15,2/1/2016 00:12:17,525,W 34 St & 11 Ave,40.755942,-74.002116,472,E 32 St & Park Ave,40.745712,-73.981948,22934,Subscriber,1990,1
7,1615,2/1/2016 00:02:45,2/1/2016 00:29:40,490,8 Ave & W 33 St,40.751551,-73.993934,473,Rivington St & Chrystie St,40.721101,-73.991925,15342,Subscriber,1966,1
8,614,2/1/2016 00:02:56,2/1/2016 00:13:10,459,W 20 St & 11 Ave,40.746745,-74.007756,237,E 11 St & 2 Ave,40.730473,-73.986724,15883,Subscriber,1991,1
9,268,2/1/2016 00:02:56,2/1/2016 00:07:25,345,W 13 St & 6 Ave,40.736494,-73.997044,285,Broadway & E 14 St,40.734546,-73.990741,16831,Subscriber,1969,2


In [15]:
# filter by neighbourhood
start_station_data = data[['start station id', 'start station name', 'start station latitude', 'start station longitude']]
end_station_data = data[['end station id', 'end station name', 'end station latitude', 'end station longitude']]

In [28]:
f = open('stations.json', 'r')
stations_json = json.loads(f.read())
stations_full = json_normalize(stations_json['stationBeanList'])

In [29]:
stations_full.columns

Index([u'altitude', u'availableBikes', u'availableDocks', u'city', u'id',
       u'landMark', u'lastCommunicationTime', u'latitude', u'location',
       u'longitude', u'postalCode', u'stAddress1', u'stAddress2',
       u'stationName', u'statusKey', u'statusValue', u'testStation',
       u'totalDocks'],
      dtype='object')

In [81]:
stations = stations_full[['id', 'latitude', 'longitude', 'stationName']]
stations.count()
stations.head()

,id,latitude,longitude,stationName
0,72,40.767272,-73.993929,W 52 St & 11 Ave
1,79,40.719116,-74.006667,Franklin St & W Broadway
2,82,40.711174,-74.000165,St James Pl & Pearl St
3,83,40.683826,-73.976323,Atlantic Ave & Fort Greene Pl
4,116,40.741776,-74.001497,W 17 St & 8 Ave


In [65]:
streetEasy_key1 = 'a00fc395e74bd46ece55d52b98b783445bcf2acc'
streetEasy_key2 = '615e0d6b82f9198a3b690baf51383e58b978f635'
streetEasy_key3 = 'fd0c7be4a56199306a34118253c55c580c5d04ca'
streetEasy_key4 = 'aac52f8de6deb59c4ac946347f5a0e7b2003f50d'
streetEasy_key5 = '8778d47f2bc849f2b6d6d0388cd4280382908578'
streetEasy_key6 = 'a1c1df657e16669aade950d84e1e5047fc850daf'

In [83]:
def get_area(row, streetEasy_key):
    lat = row['latitude']
    lon = row['longitude']
    url = 'http://streeteasy.com/nyc/api/areas/for_location?lon='+ `lon` + '&lat=' + `lat` + '&key=' + streetEasy_key + '&format=json'
    response = requests.get(url)
    neibs = response.json()
    row['city'] = neibs['city']
    row['neighborhood'] = neibs['subtitle'][3:]
    row['neighborhood_id'] = neibs['parent_id']
    row['area'] = neibs['name']
    row['area_id'] = neibs['id']
    return row

In [84]:
augmented_stations1 = stations[0:100].apply(lambda row : get_area(row, streetEasy_key1), axis=1)
augmented_stations2 = stations[100:200].apply(lambda row : get_area(row, streetEasy_key2), axis=1)
augmented_stations3 = stations[200:300].apply(lambda row : get_area(row, streetEasy_key3), axis=1)
augmented_stations4 = stations[300:400].apply(lambda row : get_area(row, streetEasy_key4), axis=1)
augmented_stations5 = stations[400:500].apply(lambda row : get_area(row, streetEasy_key5), axis=1)
augmented_stations6 = stations[500:].apply(lambda row : get_area(row, streetEasy_key6), axis=1)

In [ ]:
# final dataset with neighborhoods info of stations
augmented_stations = pd.concat([augmented_stations1, augmented_stations2, augmented_stations3, \
                                augmented_stations4, augmented_stations5, augmented_stations6])

In [97]:
pckl_file = open('augmented_stations.pickle', 'wb')
pickle.dump(augmented_stations, pckl_file)

In [ ]:
augmented_stations = pickle.load(open('augmented_stations.pickle', 'rb'))